### compareModels - A Python Jupyter Notebook to explore FEL Lamp interpolation models

In [1]:
import re
import sys  
sys.path.insert(1, '../src')
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from WhiteSpline import WhiteSpline
from NIST import NIST
from SSBUV import SSBUV, SSBUVw0 
from FELinterp import readOptronicLampData

print("FELinterp - A Python package for FEL Lamp interpolation models")

FELinterp - A Python package for FEL Lamp interpolation models


In [2]:
lampSN = 'F1711'
# lampSN = 'F1738'
# lampSN = 'F1739'
nsamples = 50

df = readOptronicLampData(lampSN)

Reading /Users/ericrehm/src/FELinterp/lamps/Optronic/F-1711/F1711_21.std
Reading /Users/ericrehm/src/FELinterp/lamps/Optronic/F-1711/F1711_k2uncertainty.dat


In [ ]:
# Chose an arbitrary reference model
RefModel = SSBUV
Models = (NIST, WhiteSpline)

# Choose and arbitrary wavelength grid for interpolation
w_interp = np.linspace(250.0, 1100.0, (1100-250)+1)  # Arbitrary wl grid

# Create the reference model
# This will be used to compute relative residuals for all other models
irr_ref = RefModel(df.wavelength.to_numpy(), df.irradiance.to_numpy(), df.uncertainty_rel.to_numpy()).model(w_interp)
print(f"{RefModel.__name__} is the reference model.")


# Each comparison model will be plotted as a separate trace
# The y-axis will show the relative residuals compared to the reference model
fig = go.Figure()
for model in Models:
    print(model.__name__)
    irr = model(df.wavelength.to_numpy(), df.irradiance.to_numpy(), df.uncertainty_rel.to_numpy()).model(w_interp)
    irr_resid = 100 * (irr - irr_ref) / irr_ref 
    fig.add_trace(go.Scatter(
        x=w_interp, y=irr_resid, mode='lines',
        name=model.__name__,
        line=dict(width=1.5)
    ))

fig.update_yaxes(range=[-0.6, 0.6], title_text='Relative Residual [%]')
fig.show()

SSBUV is the reference model.
NIST
WhiteSpline


/Users/ericrehm/src/FELinterp/notebooks/../src/SSBUV.py:89: RuntimeWarning:

divide by zero encountered in power

